In [1]:
import os
import warnings
import sys

import pandas as pd
import numpy as np

import tarfile
import urllib.request

DOWNLOAD_ROOT = "https://raw.githubusercontent.com/ageron/handson-ml2/master/"
HOUSING_PATH = os.path.join("datasets", "housing")
HOUSING_URL = DOWNLOAD_ROOT + "datasets/housing/housing.tgz"

from sklearn.compose import ColumnTransformer
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import OneHotEncoder
from sklearn.pipeline import Pipeline
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression, RidgeCV, LassoCV, ElasticNetCV
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score

import mlflow
import mlflow.sklearn

warnings.filterwarnings("ignore")

In [2]:
def fetch_housing_data(housing_url=HOUSING_URL, housing_path=HOUSING_PATH):
    if not os.path.isdir(housing_path):
        os.makedirs(housing_path)
    tgz_path = os.path.join(housing_path, "housing.tgz")
    urllib.request.urlretrieve(housing_url, tgz_path)
    housing_tgz = tarfile.open(tgz_path)
    housing_tgz.extractall(path=housing_path)
    housing_tgz.close()

In [3]:
def load_housing_data(housing_path=HOUSING_PATH):
    fetch_housing_data()
    csv_path = os.path.join(housing_path, "housing.csv")
    return pd.read_csv(csv_path)

In [4]:
# mlflow server --backend-store-uri mlruns/ --default-artifact-root mlruns/
remote_server_uri = "http://127.0.0.1:5000/" # set to your server URI
mlflow.set_tracking_uri(remote_server_uri)  # or set the MLFLOW_TRACKING_URI in the env

In [5]:
mlflow.tracking.get_tracking_uri()

'http://127.0.0.1:5000/'

In [6]:
exp_name = "LinearRegression"
mlflow.set_experiment(exp_name)

<Experiment: artifact_location='mlruns/1', experiment_id='1', lifecycle_stage='active', name='LinearRegression', tags={}>

## MLFlow Pipeline

- User
> LR
> Lasso
> Ridge
> ElastiNet

In [7]:
# column index
rooms_ix, bedrooms_ix, population_ix, households_ix = 3, 4, 5, 6

class CombinedAttributesAdder(BaseEstimator, TransformerMixin):
    def __init__(self, add_bedrooms_per_room=True): # no *args or **kargs
        self.add_bedrooms_per_room = add_bedrooms_per_room
    def fit(self, X, y=None):
        return self  # nothing else to do
    def transform(self, X):
        rooms_per_household = X[:, rooms_ix] / X[:, households_ix]
        population_per_household = X[:, population_ix] / X[:, households_ix]
        if self.add_bedrooms_per_room:
            bedrooms_per_room = X[:, bedrooms_ix] / X[:, rooms_ix]
            return np.c_[X, rooms_per_household, population_per_household,
                         bedrooms_per_room]
        else:
            return np.c_[X, rooms_per_household, population_per_household]

In [8]:
def cleanse_data(data):
    data_num = data.select_dtypes(include=[np.number])
    data["income_cat"] = pd.cut(data["median_income"],
                               bins=[0., 1.5, 3.0, 4.5, 6., np.inf],
                               labels=[1, 2, 3, 4, 5])
    
    num_pipeline = Pipeline([
        ('imputer', SimpleImputer(strategy="median")),
        ('attribs_adder', CombinedAttributesAdder()),
        ('std_scaler', StandardScaler()),
    ])

    num_attribs = list(data_num.columns.values)
    cat_attribs = ["ocean_proximity"]

    full_pipeline = ColumnTransformer([
            ("num", num_pipeline, num_attribs),
            ("cat", OneHotEncoder(), cat_attribs),
        ])
    num_attribs_col = num_attribs + ['rooms_per_household', 'population_per_household', 'bedrooms_per_room']
    cat_attribs_col = ['ocean_proximity' + str(i) for i in range(1, 6)]

    data_prepared = full_pipeline.fit_transform(data)
    data_prepared = pd.DataFrame(data_prepared, columns=num_attribs_col+cat_attribs_col)
    
    return data_prepared

In [9]:
def eval_metrics(actual, pred):
    # compute relevant metrics
    rmse = np.sqrt(mean_squared_error(actual, pred))
    mae = mean_absolute_error(actual, pred)
    r2 = r2_score(actual, pred)
    return rmse, mae, r2

def load_data(data_path = HOUSING_PATH):
    data = load_housing_data(data_path)
    
    # Cleansing and one-hot encoding the data
    data = cleanse_data(data)

    # Split the data into training and test sets. (0.75, 0.25) split.
    train, test = train_test_split(data)

    # The predicted column is "quality" which is a scalar from [3, 9]
    train_x = train.drop(["median_house_value"], axis=1)
    test_x = test.drop(["median_house_value"], axis=1)
    train_y = train[["median_house_value"]]
    test_y = test[["median_house_value"]]
    return train_x, train_y, test_x, test_y

def train(model='linear'):
    # train a model with given parameters
    np.random.seed(40)

    # Read the wine-quality csv file (make sure you're running this from the root of MLflow!)
    train_x, train_y, test_x, test_y = load_data()
    
    # Useful for multiple runs (only doing one run in this sample notebook)    
    with mlflow.start_run():
        # Execute LinearRegression
        model = model.lower()
        model_dict = {'linear': LinearRegression, 'ridge': RidgeCV, 'lasso': LassoCV, 'elastinet':ElasticNetCV}
        lr = model_dict[model]()
        lr.fit(train_x, train_y)

        # Evaluate Metrics
        predicted_qualities = lr.predict(test_x)
        (rmse, mae, r2) = eval_metrics(test_y, predicted_qualities)

        # Print out metrics
        print(f"Model: {model.title()}")
        print("  RMSE: %s" % rmse)
        print("  MAE: %s" % mae)
        print("  R2: %s" % r2)

        # Log parameter, metrics, and model to MLflow
        mlflow.log_metric(key="rmse", value=rmse)
        mlflow.log_metrics({"mae": mae, "r2": r2})
        mlflow.log_artifact(HOUSING_PATH)
        print("Save to: {}".format(mlflow.get_artifact_uri()))
        
        mlflow.sklearn.log_model(lr, "model")

In [10]:
train()

Model: Linear
  RMSE: 0.5988452034985938
  MAE: 0.43367042256325444
  R2: 0.6488279212507495
Save to: mlruns/1/c543405f8032422a839faa6fdfbaa14c/artifacts


In [11]:
train('Ridge')

Model: Ridge
  RMSE: 0.5988975922258075
  MAE: 0.433724689198381
  R2: 0.6487664754451795
Save to: mlruns/1/bd1311f694254368809d9f719d8fe537/artifacts


In [12]:
train('Lasso')

Model: Lasso
  RMSE: 0.5990449839953267
  MAE: 0.4338084869590932
  R2: 0.648593573427638
Save to: mlruns/1/7ba1a6521a7a4cf8b455c87d00b40dc4/artifacts


In [13]:
train('ElastiNet')

Model: Elastinet
  RMSE: 0.5991188259615581
  MAE: 0.4337806192860862
  R2: 0.6485069350566772
Save to: mlruns/1/e9b222691b9047deb1ecb0c4a60a051c/artifacts
